# [Pandas](https://pandas.pydata.org/)
According to the pandas website, pandas helps fill the gap between data munging and preparation and data analysis, "enabling you to carry out your entire data analysis workflow in Python without having to switch to a more domain specific language like R."

Like in R, the main structure in Pandas is a data frame.

In [ ]:
import pandas as pd # for data frames, reading and writing data
from matplotlib import pyplot as plt
# import psycopg2 # for connecting to a postgres database - THIS LIBRARY ISN'T LOADED
import numpy as np # using this to create a range of floats
from math import sqrt

# the next line is so that the matplot lib plots show up in the notebook cell
%matplotlib inline

## Pandas Basics
* Create a data frame from scratch
* Adding/removing columns
* Descriptive information

Unlike NumPy arrays and matrices, a Pandas data frame can hold different data types. However, a data frame is made up of `Pandas.Series` (columns) which must all be of the same data type. Let's create a dataframe from scratch with a few columns of different data types.

*Create a data frame with three columns:*
* 'numbers' (integers)
* 'floats' (floats)
* 'names' (strings)*

In [ ]:
# use pd.DataFrame to create the data frame. 
# You can create the data fields in a dictionary before hand, or directly in the call to pd.DataFrame
# The dictionary should have the field name as the 'key' and the list of values as the 'value'
df = 

# View The resulting dataframe
df

Add a calculated column:

In [ ]:
# Add a calculated column as the product of the numbers and floats: 
df['calc_col'] = 

#View the resulting dataframe
df

Let's add some missing data so that we can look at how pandas treats it and how to find it when loading data sets later.

In [ ]:
# Create a new row as a dictionary and use np.nan for missing values. 
new_row = 

# Append the new row to our data frame with df.append(new_row)
# Use "ignore_index=True" when appending
df = 

# Reset the index to the names column after appending
df.index = 

#View the resulting dataframe
df

### Data Frame Descriptive Info
* Column names
* Length
* Missing data

We can use the `describe` method on a data frame to get some basic statistics on the column. The default is to only include numerical columns. Try it with and without inlclude='all' to see the different versions of the results. 

In [ ]:
df.describe()

In [ ]:
df.describe(include='all')

In [ ]:
print("Numbers:\nmean: {:.2f}\nstd: {:2f}".format(df.numbers.mean(), 
                                                  df.numbers.std()))

## Day to day Pandas
Now that we've looked at some of the Pandas basics, here are some things that seem to come up regularly when using Pandas as a data engineering tool.

* Reading/Writing files - xlsx, csv
* Subsetting and merging data frames
* Plotting

## Load Data from a Database
We'll use some data from the Twitter work we've been doing for most of the analysis. I'll start by pulling data from our Postgres database on AWS. You cannot connect to this database without someone adding your IP address to the security group on AWS, but this will show you how to pull data from a database.

NOTE: in order to connect to the Postgres database on AWS as I do in the examples below, you'll need the psycopg2 library.

Let's pull 200 tweets from each topic to get a good mix. Put each into a pandas DataFrame and merge them all together. Finally, save them to Excel for the sample data for you to use. 

BEWARE - when pulling twitter ids (or any very large integers) into Excel, Excel tends to round them ton 15 places, losing 3 digits and making joins and merges break. Saving as csv avoids this problem, as long as you don't then view the csv in Excel.

NOTE: I'm commenting all of this database stuff out, since you won't be able to connect anyway. Leaving the cells for my reference and yours, so you can see how database connections work with pandas.

In [1]:
# # Database Connection parapeters
# hostname = 'ditwitter.c6rgtnn1vfuu.us-east-1.rds.amazonaws.com'
# username = 'ditwitter_sa'
# pwd = 'ThriventTwitter'
# database = 'ditwitter'

# # Connect
# conn = psycopg2.connect( host=hostname, user=username, password=pwd, dbname=database )

In [2]:
# # First let's get a list of topics:
# SQL = """SELECT DISTINCT topics.* 
#         FROM topics
#         INNER JOIN models ON md_tp_id = tp_id"""

# topics_df = pd.read_sql(SQL, con=conn)
# topics_df

### Tweet Data

In [3]:
# # Create an empty data frame too hold the tweets we're going to collect
# tweets_df = pd.DataFrame()

# # Loop through all the active topics and grab a block of tweets, then merge with the tweets_df
# block_size = 200

# for tp_id in topics_df['tp_id']:
#     SQL = """SELECT t.*, tp_name 
#     FROM tweets t
#     INNER JOIN tweet_scores ts ON ts.ts_tweet_id = t.tweet_id
#     INNER JOIN models m ON m.md_id = ts.ts_md_id
#     INNER JOIN topics tp ON tp.tp_id = m.md_tp_id
#     WHERE tp_id = {}
#     LIMIT {}""".format(tp_id, block_size)
    
#     tweet_block = pd.read_sql(SQL, conn)
# #     print("pulled {} for topic_id: {}.".format(len(tweet_block), tp_id))
#     tweets_df = tweets_df.append(tweet_block)

# tweets_df.head()

### User Data
Let's pull the user data for all of these records. To do that, we'll need to build a "WHERE" clause that has all the unique user_ids from our tweets dataframe. We'll need to convert the values to strings, then separate them by commas. 

In [4]:
# # Build the list of user_ids
# sep = ','
# users_string = sep.join(tweets_df['user_id'].astype(str))

# SQL = "SELECT * FROM users WHERE id in ({})".format(users_string)
# users_df = pd.read_sql(SQL, conn)
# print(len(users_df))
# users_df.head()

## Save to Excel
Now that I've pulled the data for the examples, I'll save it to Excel for easy distribution. This is where we have to do something to get around the Excel string conversion issue. We don't care about tweet_ids, but we DO care about user_ids, since that's the field that we'll later join these two datasets on. To avoid the conversion issue, we'll add a column to the tweets_data that converts the user_id to string. We already have this column in the user data as `id_str`.

*NOTE: I've commented out the save-to-excel code, since I've since added other data to the sample_data.xlsx file that I don't want overwritten. Leaving it in here for reference.*

In [ ]:
# tweets_df['user_id_str'] = tweets_df['user_id'].astype(str)

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
# writer = pd.ExcelWriter('sample_data.xlsx', engine='xlsxwriter')

# # Write each dataframe to a different worksheet.
# tweets_df.to_excel(writer, sheet_name='tweet_data', index=False)
# users_df.to_excel(writer, sheet_name='user_data', index=False)
# writer.save()


### Read Data from Excel
I'm using pandas to read in the data file from excel. If the file is located in the same directory as the notebook, this will work. Otherwise, add the path to the file to the filename. Pandas will automatically infer data types, column numbers and rownumbers from the data. There are quite a few different arguments that you can pass to this function to control what is loaded and how. The following cell will bring up the docstring for this function that has explanations for all of the options.

In [ ]:
filename = 'sample_data.xlsx'
# use pd.read_excel to read in the 'tweets_classified' sheet
t_data = 

# review the data that you got
t_data.head()

### Export dataframe to tab delimited file
Now that we have some data to work with, we can export it to a tab-delimited file. After exporting, we'll remove the data frame and reload it from the csv file.
* setting the sep argument to '\t' makes it tab separated. Default is comma separated
* setting the index=False prevents it from writing out the row numbers as a column, creating an exraneous column.

In [ ]:
export_filename = 'sample_data.csv'
# use `.to_csv` to export the data to a file
# Export to a file

# destroy the object by setting it to None in case it is using a lot of memory
t_data =

Read the file back in to make sure everything worked as expected.

In [ ]:
# read in the file with pd.read_csv
t_data = 

# View the file
t_data.head()

Let's get an idea of what's in this dataframe - We know it has texts from different topics. Let's see how many from each are in there:

In [ ]:
t_data['topic'].value_counts()

In [ ]:
t_data['class'].value_counts()

Since we want the `class` variable to be binary, we have some data clean-up to do here. At some point I started using 2 for negatives, since it was easier on the keyboard than 0! Let's replace all of those 2s with 0 to make class truly binary.

In [ ]:
t_data.loc[t_data['class']==2,'class'] = 0 
t_data['class'].value_counts()

In [ ]:
# Check data types
t_data.dtypes

In [ ]:
# Get some descriptive data from this dataframe
t_data.describe(include='all')

### Subsetting
Subsetting dataframes with Pandas is very similar to subsetting in R. Since the sample data has data from 5 different topics, let's pull out two topics and make them separate data frames.

Unlike R, when subsetting with Pandas you have to use `loc` or `iloc` before adding in the subset parameters. 
* `loc` is used when you have a criteria based on the values in a column or multiple columns
* `iloc` will give you the values from a numeric position in the dataframe. For example, if you wanted the first 10 rows of the data frame, you'd do the following:

*NOTE: Unlike R, Python is zero-based, so lists and indexes start at zero, rather than one.* 
#### Subsetting with `iloc`

In [ ]:
# First 10 rows
t_data.iloc[ ]

#### Subsetting with `loc`
When referencing columns in pandas, you can use either dataframe.column_name or dataframe['column name']. They should work the same way. Sometimes, maybe based on the column name itself, the .column_name doesn't work. ['column_name'] seems to be more reliable. In this dataframe, I had this issue with the 'class' column.

In [ ]:
moving_df = t_data.loc[ ]
moving_df.head()

In [ ]:
marriage_df = t_data.loc[ ]
marriage_df.head()

#### Multiple subset criteria
This works the same way that subsetting in R does. Let's find all of the Marriage and Moving tweets where the Class==1. A common error when subsetting is: `The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().` If you get that, make sure you are using `&` and `|` for and/or operators. If it's still an issue, check parentheses - it seems like it needs more than necessary for the subset to work!

In [ ]:
# use .loc and the criteria that (topic=='Marriage' & class==1) or (topic=='Moving' & class==1)
subset2 = t_data.loc[(  &  ) |
                     (( &  )]
subset2.head()

### Merging Data Frames
Now that we have two separate data frames for Marriage and Moving, let's merge them together and see if the number of class==1 matches our subset above. There are a lot of options when merging data frames - similar to joins with data tables. The [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) is pretty helpful as is StackOverflow, of course.

This first example is just a combination of two dataframes... no index to match on. 

In [ ]:
# Merge the marriage_df and moving_df as an outer-join. You can pick either one and .merge with the other data frame
merged_df = marriage_df.merge(  )
print('Marrige data frame: {}'.format(len(marriage_df)))
print('Moving data frame: {}'.format(len(moving_df)))
print('Merged data frame: {}'.format(len(merged_df)))

In [ ]:
# Does our count of positive tweets match between the subset and the merged data?
len(subset2) == len(merged_df.loc[merged_df['class']==1])

We need some different data to show how to merge on different keys. Our sample file has data for this too in the `tweet_data` and `user_data` sheets.

In [ ]:
# use pd.read_excel to read in the 'tweet_data' and 'user_data' sheets from the sample data spreadsheet
tweet_df = 
tweet_df.head()

In [ ]:
user_df = 
user_df.head()

Now that we have a handful of tweets, we want to merge the tweet data with the user data to append specific user columns to the tweet data. Let's only grab a few columns from each data frame to keep it easy to read. We can select a subset of columns with no other criteria with `dataframe[[list of columns]]`.

The `how` parameter of the merge works like a join, defining what rows to keep when there isn't a match in both dataframes. It defaults to an inner join. In this case I want to keep all of the tweets, even if we don't have a user record, so I'm using `how=left` since the first table in the merge (the left one) is the tweet_df.

NOTE: join on tweet_df.user_id_str = user_df.id_str, to avoid any truncation of the long integers that may have happend in exporting to Excel!

### Merge using a unique match key.

In [ ]:
# Merge ['tweet_id', 'created_at', 'user_id_str', 'text'] from the tweet_df with
# ['id_str', 'name','screen_name', 'followers_count'] from the user_df
# Left outer join on tweet_df.user_id_str = user_df.id_str

merged_tweets = tweet_df[ ].merge(
    user_df[ ],
    left_on= ,
    right_on= ,
    how= )


merged_tweets.head()

### Bin a continuous variable into a new variable.
Since we have a bunch of users, let's bin their followers_count into equal width bins. This was a new one for me, but there is a handy pandas function for it, similar to R, called [pandas.cut](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html)

In [ ]:
merged_tweets['binned_followers'] = pd.cut(merged_tweets['followers_count'], bins=5, labels = ['very_low',
                                                                                               'low',
                                                                                               'medium',
                                                                                               'high',
                                                                                              'very_high'])
merged_tweets['binned_followers'].value_counts()

### Binning with an equal number of members
Cutting the followers_count into equal sized bins wasn't very helpful, since there are two users with so many followers that the ranges become unuseful.  

More useful may be to use quartiles. For that, we'll have to calculate the quartiles ahead of time, then pass them into the `cut` function as the `squence of scalars`.

In [ ]:
bins = 5
# use pandas.quantile function and np.linspace to generate the cutoff values for the cut-function.
cutoffs = list(merged_tweets['followers_count'].quantile(np.linspace(0,1,bins+1)))

# create some labels for our new, binned column
labels = ['Q'+str(x) for x in range(1,bins+1)]

# cut the data based on the cutoffs
merged_tweets['quartile_followers'] = pd.cut(merged_tweets['followers_count'], cutoffs, labels = labels)

# use 'value_counts' again to check if it worked


Let's see how that worked out. We'll look at the mean, median and median and standard deviation for each quartile of follower_counts. We can use the groupby function in pandas to get these aggregates.

In [ ]:
# Create a groupby object from merged_tweets
# groupby 'quartile_followers'
# aggregate the 'followers_count' column with the following funcitions: 
# count, min, max, mean, median, std

merged_tweets.groupby( )[ ].agg([ ])

### Macro variable for data selection
From Corinne: *In SAS we frequently create a list of variables by putting them in a macro variable that we use for data exploration and variable selection so that we can perform the necessary tasks for all variables easily.*

Using the user_df data frame, let's pick a subset of data for our macro variable. The macro variable will be a list with the column names we want.

In [ ]:
# Show the list of all columns
list(user_df.columns)

In [ ]:
# Create our subset variable - let's pick all the numerical fields
col_subset = [
    
    
    
]

# Now we can use this variable to select from the data:
user_df[col_subset].head()

### Crosstab, Pivots and GroupBy
Back to our classified tweet data, we have different topics (multiple) and we have classification (binary). We can create a two-way frequency table showing the number of each class in each topic. We can use panads.crosstab to get to this result.

#### Crosstab

In [ ]:
two_way = pd.crosstab(t_data['topic'],t_data['class'])
two_way

#### Pivot Table
You can also use the pivot_table function to get to the same results.

In [ ]:
two_way_pivot = t_data.pivot_table(index='topic', columns = 'class', aggfunc=len)

Both of these results have a multi-part index, making it a little complicated to subset the results. Since it has a multiple index, you have to pass values or criteria for both components of the index.

In [ ]:
two_way.loc[(['Marriage','Moving'],[0,1])]

In [ ]:
two_way_pivot.loc[['Marriage','Moving']]

This pivot approach seems to be more difficult to subset, as doesn't like my second part of the index.

Let's revisit our GroupBy table from the previous section and look at a pivot version of it. Here we are looking to get aggregate data for the different quartiles that we created based on follower_count:

In [ ]:
grouped_df = merged_tweets.groupby('quartile_followers')['followers_count'].agg(['count','min','max','mean', 'median', 'std'])
grouped_df

### More Pivoting
Similar to pivot tables in Excel, Pandas creates a hierarchy based on the pivot values and then applies one or multiple aggregate functions to the values not included in the index.

In [ ]:
pivot_data = user_df[['followers_count', 'friends_count', 'favourites_count', 'lang', 'statuses_count', 'time_zone', 'state']]
pivoted = pivot_data.pivot_table(index = ['time_zone', 'state'], aggfunc=['count','mean'])
pivoted.columns

Selecting data from a dataframe with a multi-level index is more complicated than a simple data frame. This [reference](https://pandas.pydata.org/pandas-docs/stable/advanced.html) can help. Let's select two timezones from the pivot table that we just created and the mean of two of the columns.

In [ ]:
pivoted.loc[(['Eastern Time (US & Canada)','Central Time (US & Canada)'], 'mean')][['friends_count', 'favourites_count']]

In [ ]:
pivoted2 = pivot_data.pivot_table(index = ['time_zone'], aggfunc=['count','mean'])
pivoted2.loc[['Eastern Time (US & Canada)','Central Time (US & Canada)'], 'mean'][['friends_count', 'favourites_count']]

### How to generate a summary of interval/continuous/numeric variables including
* Basic statistics like the mean, median, percentiles, standard deviation, etc.
* Confidence intervals around the mean

A quick way to get to some of this information is with the `describe` function on a dataframe. By default, this will only describe numeric variables:

In [ ]:
merged_tweets.describe()

If we add `include='all'` we'll get descriptive date on the rest of the columns, with a bunch of 'NaN' for irrelevant statistics.

In [ ]:
merged_tweets.describe(include='all')

Since this gives you the standard deviation for numeric fields, you can use that to create confidence intervals as needed. Here's how you can pull values out of this function - let's say we want the standard deviation for the followers_count:

In [ ]:
# set the description to a data frame variable, then pull the value as a subset
desc = merged_tweets.describe(include='all')
desc.loc['std','followers_count']

In [ ]:
# or call the function and pull the value directly from the results, if there's no other need for that data.
merged_tweets.describe().loc['std','followers_count']

In [ ]:
# Double checking that it's right using the standard deviation function (std)
merged_tweets['followers_count'].std()